## План
- Строим матрицу
- Телепортация
- Нормализуем (сумма строки = 1)
- Домножаем на случайный вектор, пока не сойдется

In [1]:
import os
import urllib
from bs4 import BeautifulSoup
import numpy as np
from tqdm import tqdm

In [2]:
filenames = os.listdir("raw")

In [3]:
len(filenames)

2379

In [4]:
topics = {}
for i, file in enumerate(filenames):
    topics[file[:-5]] = i

In [5]:
matrix = np.zeros((len(filenames), len(filenames)))

In [6]:
for file in tqdm(filenames):
    with open(f"raw/{file}", encoding='utf-8') as f:
        cur_topic = file[:-5]
        bs = BeautifulSoup(f.read(), "html.parser")
        all_a = bs.find_all("a")
        for link in all_a:
            if not 'href' in link.attrs:
                continue
            href = urllib.parse.unquote(link['href'])
            if href.startswith("/wiki/") and href[6:] in topics:
                matrix[topics[cur_topic]][topics[href[6:]]] = 1

100%|██████████████████████████████████████| 2379/2379 [05:19<00:00,  7.45it/s]


In [8]:
for i in matrix:
    for j in range(len(i)):
        if i[j] == 0:
            i[j] = 0.001

In [9]:
row_sums = matrix.sum(axis=1)

In [10]:
norm_matrix = matrix / row_sums[:, np.newaxis]

In [11]:
vector = np.array([0.2]*len(filenames))

In [12]:
for _ in range(100):
    vector = norm_matrix.transpose().dot(vector)

In [13]:
np.take(vector, [1, 2])

array([0.90675362, 0.75596967])

In [14]:
rank = {}

In [15]:
for i, value in enumerate(vector):
    rank[i] = value

In [23]:
def page_rank(rank, n=10, word=None):
    if word is not None:
        new_topics = [ind for topic, ind in topics.items() if word.lower() in topic.lower()]
    
    if word is not None:
        new_rank = {}
        for ind in new_topics:
            new_rank[ind] = rank[ind]
    else:
        new_rank = rank.copy()
        
    items = sorted(new_rank.items(), key=lambda t: t[1])[::-1][:n]
    for index, value in items:
        print(f"{filenames[index][:-5]}: {value}")

In [17]:
page_rank(rank=rank)

Английский_язык: 6.7102441063494815
США: 4.666826841990531
Франция: 3.891332065960341
Великобритания: 3.755379111713282
Германия: 3.045402505156282
Соединённые_Штаты_Америки: 2.9824853687421458
Италия: 2.411497480410388
СССР: 2.385908911034201
Викисклад: 2.308323432805024
Латинский_язык: 2.305412331063708


In [24]:
page_rank(rank=rank, word='армстронг')

Армстронг,_Луи: 0.21913073924572785
Армстронг,_Нил_Олден: 0.08204167651131537
Армстронг,_Нил: 0.07772736651558698
Армстронг,_Билли_Джо: 0.06033081219855766
Армстронг,_Алан: 0.05467384689837547
Армстронг_(фамилия): 0.054339929130478144
Армстронг,_Эдвин: 0.048724294245225605
Армстронг,_Лэнс: 0.047655826559074285
Билли_Джо_Армстронг: 0.04497463456555407
Армстронг,_Крэйг: 0.03901933675387252
